# Test Model on VRPLib

In this notebook, we will test the trained model's performance on the VRPLib benchmark. We will use the trained model from the previous notebook.

[VRPLIB](http://vrp.galgos.inf.puc-rio.br/index.php/en/) is a collection of instances related to the CVRP, which is a classic optimization challenge in the field of logistics and transportation. 

<a href="https://colab.research.google.com/github/ai4co/rl4co/blob/main/examples/datasets/2-test-on-cvrplib.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

## Before we start

To use the VRPLib, we strongly recomment to use the Python `vrplib` tool:

[VRPLib](https://github.com/leonlan/VRPLIB) is a Python package for working with Vehicle Routing Problem (VRP) instances. This tool can help us easily load the VRPLib instances and visualize the results.

## Installation

Uncomment the following line to install the package from PyPI. Remember to choose a GPU runtime for faster training!

> Note: You may need to restart the runtime in Colab after this


In [68]:
# !pip install rl4co[graph] # include torch-geometric

## NOTE: to install latest version from Github (may be unstable) install from source instead:
# !pip install git+https://github.com/ai4co/rl4co.git

: 

In [42]:
# Install the `vrplib` package
# !pip install vrplib

## Imports

In [43]:
%load_ext autoreload
%autoreload 2

import os
import re
import torch
import vrplib

from rl4co.envs import TSPEnv, CVRPEnv
from rl4co.models.zoo.am import AttentionModel, AttentionModelPolicy
from rl4co.utils.trainer import RL4COTrainer
from rl4co.utils.decoding import get_log_likelihood
from rl4co.models.zoo import EAS, EASLay, EASEmb, ActiveSearch

from tqdm import tqdm
from math import ceil
from einops import repeat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load a trained model

In [44]:
# Load from checkpoint; alternatively, simply instantiate a new model
# Note the model is trained for CVRP problem
checkpoint_path = "../cvrp-20.ckpt" # modify the path to your checkpoint file

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load checkpoint
# checkpoint = torch.load(checkpoint_path)

# lit_model = AttentionModel.load_from_checkpoint(checkpoint_path, load_baseline=False)

env = CVRPEnv()
policy = AttentionModelPolicy(env_name=env.name)

# policy, env = lit_model.policy, lit_model.env
policy = policy.to(device)

## Download vrp problems

In [45]:
problem_names = vrplib.list_names(low=50, high=200, vrp_type='cvrp') 

instances = [] # Collect Set A, B, E, F, M datasets
for name in problem_names:
    if 'A' in name:
        instances.append(name)
    elif 'B' in name:
        instances.append(name)
    elif 'E' in name:
        instances.append(name)
    elif 'F' in name:
        instances.append(name)
    elif 'M' in name and 'CMT' not in name:
        instances.append(name)

# Modify the path you want to save 
# Note: we don't have to create this folder in advance
path_to_save = './vrplib/' 

try:
    os.makedirs(path_to_save)
    for instance in tqdm(instances):
        vrplib.download_instance(instance, path_to_save)
        vrplib.download_solution(instance, path_to_save)
except: # already exist
    pass 

/home/botu/mambaforge/envs/rl4co/lib/python3.11/site-packages/vrplib/download/list_names.py:32: DeprecationWarning: The function 'list_names' is deprecated and will be removed in the next major version (vrplib v2.0.0).
  warnings.warn(msg, DeprecationWarning)


In [46]:
# Utils function
def normalize_coord(coord:torch.Tensor) -> torch.Tensor:
    x, y = coord[:, 0], coord[:, 1]
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    
    x_scaled = (x - x_min) / (x_max - x_min) 
    y_scaled = (y - y_min) / (y_max - y_min)
    coord_scaled = torch.stack([x_scaled, y_scaled], dim=1)
    return coord_scaled 

## Test the greedy

In [57]:
from tensordict import TensorDict

problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))

# problem
coords = torch.tensor(problem['node_coord']).float()
coords_norm = normalize_coord(coords)
demand = torch.tensor(problem['demand'][1:]).float()
capacity = problem['capacity']
n = coords.shape[0]
td = TensorDict({
    'depot': coords_norm[0,:],
    'locs': coords_norm[1:,:],
    'demand': demand / capacity, # normalized demand
    'capacity': capacity, # original capacity, not needed for inference
})
td = td[None] # add batch dimension, in this case just 1

# start
td_reset =  env.reset(td.to(device)).to(device)    

In [51]:
out = policy(td_reset, env)

## Test the Augmentation

In [67]:
# Import augmented utils
from rl4co.data.transforms import (
    StateAugmentation as SymmetricStateAugmentation)
from rl4co.utils.ops import batchify, unbatchify

num_augment = 100
augmentation = SymmetricStateAugmentation(num_augment=num_augment)

for instance in instances:
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))

    # problem
    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords)
    demand = torch.tensor(problem['demand'][1:]).float()
    capacity = problem['capacity']
    n = coords.shape[0]
    td = TensorDict({
        'depot': coords_norm[0,:],
        'locs': coords_norm[1:,:],
        'demand': demand / capacity, # normalized demand
        'capacity': capacity, # original capacity, not needed for inference
    })
    td = td[None] # add batch dimension, in this case just 1
    td =  env.reset(td.to(device))   
    
    # Get the solution from the policy
    with torch.inference_mode():
        out = policy(
            td.clone(), env, decode_type='sampling', num_starts=128, select_best=True, multisample=True
        )

    # reward = env.get_reward(td, out['actions'])
    # reward = unbatchify(reward, num_augment)
    
    # set back to original coordinates
    td["locs"] = coords[None] # note: these contain already the depot
    print(out["actions"].shape)
    reward = env.get_reward(td, out['actions'])
    cost = ceil(-1 * torch.max(reward).item())

    # Load the optimal cost
    solution = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = solution['cost']

    # Calculate the gap and print
    gap = (cost - optimal_cost) / optimal_cost
    print(f'Problem: {instance:<15} Cost: {cost:<10} Optimal Cost: {optimal_cost:<10}\t Gap: {gap:.2%}')

torch.Size([1, 61])
Problem: A-n53-k7        Cost: 2635       Optimal Cost: 1010      	 Gap: 160.89%
torch.Size([1, 61])
Problem: A-n54-k7        Cost: 2966       Optimal Cost: 1167      	 Gap: 154.16%
torch.Size([1, 65])
Problem: A-n55-k9        Cost: 2793       Optimal Cost: 1073      	 Gap: 160.30%
torch.Size([1, 74])
Problem: A-n60-k9        Cost: 3225       Optimal Cost: 1354      	 Gap: 138.18%
torch.Size([1, 70])
Problem: A-n61-k9        Cost: 2799       Optimal Cost: 1034      	 Gap: 170.70%
torch.Size([1, 71])
Problem: A-n62-k8        Cost: 3412       Optimal Cost: 1288      	 Gap: 164.91%
torch.Size([1, 76])
Problem: A-n63-k9        Cost: 3735       Optimal Cost: 1616      	 Gap: 131.13%
torch.Size([1, 73])
Problem: A-n63-k10       Cost: 3143       Optimal Cost: 1314      	 Gap: 139.19%
torch.Size([1, 72])
Problem: A-n64-k9        Cost: 3391       Optimal Cost: 1401      	 Gap: 142.04%
torch.Size([1, 75])
Problem: A-n65-k9        Cost: 3364       Optimal Cost: 1174      	 Gap